In [1]:
import geopandas as gpd
import pandas as pd
from osgeo import gdal
import numpy as np
from osgeo import ogr, osr

# Rwanda

In [2]:
rwa_bridges = gpd.read_file('data/rwa-feb032023.geojson')
rwa_bridges = rwa_bridges[rwa_bridges.geometry.notnull()]
rwa_bridges = rwa_bridges.set_geometry('geometry')
rwa_bridges = rwa_bridges.to_crs(epsg=4326)

# only including bridges that are complete or under construction
# rwa_bridges = rwa_bridges[rwa_bridges['Stage'].isin(['Complete', 'Under Construction'])]

rwa_bridges

,Opportunity Name,Level 2 Government,Bridge Name,Project Code,Stage,Sub-Stage,CaseSafeID,B2P Fiscal Year,Individuals Directly Served,Population Estimate 2000m,geometry
0,Rwanda - Kirwa - 1012699,Ruhango,Kirwa,1012699,Identified,Technical,006f100000dhyeVAAQ,2018,827.0,4586.0,POINT (29.61977 -2.22162)
1,Rwanda - Kajeje - 1022230,Muhanga,Kajeje,1022230,Identified,Requested,0063Z00000hvy75QAA,2022,300.0,NaN,POINT (29.79243 -1.97174)
2,Rwanda - Nyaruhombo - 1009280,Huye,Nyaruhombo,1009280,Complete,In Service,006f100000aWV5fAAG,2023,2373.0,5115.0,POINT (29.67439 -2.41201)
3,Rwanda - Birambo - 1037670,Gicumbi,Birambo,1037670,Under Construction,,0063Z00000pof1dQAA,2023,500.0,NaN,POINT (30.13857 -1.58050)
4,Rwanda - Ruharazi - 1009311,Nyaruguru,Ruharazi,1009311,Complete,In Service,006f100000asEzNAAU,2019,NaN,7393.0,POINT (29.62627 -2.77676)
...,...,...,...,...,...,...,...,...,...,...,...
1533,Rwanda - Murunda - 1042893,Rutsiro,Murunda,1042893,Identified,Requested,0063Z00000wbTF0QAM,2024,4956.0,NaN,POINT (29.37086 -1.91510)
1534,Rwanda - Kabira 2 - 1042892,Rutsiro,Kabira 2,1042892,Identified,Requested,0063Z00000wbTEvQAM,2024,4888.0,NaN,POINT (29.37899 -1.97962)
1535,Rwanda - Kabilizi-Runga - 1042784,Nyanza,Kabilizi-Runga,1042784,Identified,Requested,0063Z00000wbMI3QAM,2024,21109.0,NaN,POINT (29.68048 -2.37512)
1536,Rwanda - Minigo - 1042823,Rubavu,Minigo,1042823,Identified,Requested,0063Z00000wbOZQQA2,2024,5084.0,NaN,POINT (29.29481 -1.69206)


in the EPSG 4326 coordinate system, 1 degree = 69 miles. We want to calculate a 2 mile buffer around each point. 2 miles / 69 miles per degree ≈ 0.0289855072463768 degrees

In [3]:
rwa_bridges['geometry'] = rwa_bridges['geometry'].buffer(0.029)

/var/folders/3v/8xxt4zw96bn2l6qwb8w7_5r40000gn/T/ipykernel_66625/816456795.py:1: UserWarning: Geometry is in a geographic CRS. Results from 'buffer' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  rwa_bridges['geometry'] = rwa_bridges['geometry'].buffer(0.029)


Population data was procured from World Pop 2020 data. This was converted from raster to vector points using QGIS raster pixels to points. 

In [4]:
# read in rwa_pop_2020.geojson
rwa_pop = gpd.read_file('unsynced-data/rwa-pop-2020.geojson')
# rename pop- population
rwa_pop = rwa_pop.rename(columns={'pop-': 'population'})

In [5]:
# calculate the sum of rwa_pop points within each of the rwa_bridges polygons and add this to a new column in rwa_bridges
rwa_bridges['pop_sum'] = rwa_bridges['geometry'].apply(lambda x: rwa_pop['population'].loc[rwa_pop.within(x)].sum())
rwa_bridges['pop_sum'] = rwa_bridges['pop_sum'].astype(int)

In [6]:
rwa_bridges

,Opportunity Name,Level 2 Government,Bridge Name,Project Code,Stage,Sub-Stage,CaseSafeID,B2P Fiscal Year,Individuals Directly Served,Population Estimate 2000m,geometry,pop_sum
0,Rwanda - Kirwa - 1012699,Ruhango,Kirwa,1012699,Identified,Technical,006f100000dhyeVAAQ,2018,827.0,4586.0,"POLYGON ((29.64877 -2.22162, 29.64863 -2.22446...",13492
1,Rwanda - Kajeje - 1022230,Muhanga,Kajeje,1022230,Identified,Requested,0063Z00000hvy75QAA,2022,300.0,NaN,"POLYGON ((29.82143 -1.97174, 29.82129 -1.97458...",15984
2,Rwanda - Nyaruhombo - 1009280,Huye,Nyaruhombo,1009280,Complete,In Service,006f100000aWV5fAAG,2023,2373.0,5115.0,"POLYGON ((29.70339 -2.41201, 29.70325 -2.41485...",17644
3,Rwanda - Birambo - 1037670,Gicumbi,Birambo,1037670,Under Construction,,0063Z00000pof1dQAA,2023,500.0,NaN,"POLYGON ((30.16757 -1.58050, 30.16743 -1.58335...",8871
4,Rwanda - Ruharazi - 1009311,Nyaruguru,Ruharazi,1009311,Complete,In Service,006f100000asEzNAAU,2019,NaN,7393.0,"POLYGON ((29.65527 -2.77676, 29.65513 -2.77960...",17113
...,...,...,...,...,...,...,...,...,...,...,...,...
1533,Rwanda - Murunda - 1042893,Rutsiro,Murunda,1042893,Identified,Requested,0063Z00000wbTF0QAM,2024,4956.0,NaN,"POLYGON ((29.39986 -1.91510, 29.39972 -1.91795...",19743
1534,Rwanda - Kabira 2 - 1042892,Rutsiro,Kabira 2,1042892,Identified,Requested,0063Z00000wbTEvQAM,2024,4888.0,NaN,"POLYGON ((29.40799 -1.97962, 29.40785 -1.98247...",21970
1535,Rwanda - Kabilizi-Runga - 1042784,Nyanza,Kabilizi-Runga,1042784,Identified,Requested,0063Z00000wbMI3QAM,2024,21109.0,NaN,"POLYGON ((29.70948 -2.37512, 29.70934 -2.37796...",15375
1536,Rwanda - Minigo - 1042823,Rubavu,Minigo,1042823,Identified,Requested,0063Z00000wbOZQQA2,2024,5084.0,NaN,"POLYGON ((29.32381 -1.69206, 29.32367 -1.69490...",86686


In [7]:
rwa_bridges.to_file('data/rwa-population-2mi.geojson', driver='GeoJSON')
rwa_bridges.to_csv('data/rwa-population-2mi.csv')

# Uganda

In [8]:
uga_bridges = gpd.read_file('data/uga-may242023.geojson')
uga_bridges = uga_bridges[rwa_bridges.geometry.notnull()]
uga_bridges = uga_bridges.set_geometry('geometry')
uga_bridges = uga_bridges.to_crs(epsg=4326)

# only including bridges that are complete or under construction
# uga_bridges = uga_bridges[uga_bridges['Stage'].isin(['Complete', 'Under Construction'])]

uga_bridges

,Opportunity Name,Level 2 Government,Bridge Name,Project Code,Stage,Sub-Stage,B2P Fiscal Year,Individuals Directly Served,Population Estimate 2000m,geometry
0,- Kikambajo - 1037598,Hoima,Kikambajo,1037598,Identified,NaN,2021,NaN,NaN,POINT (31.33357 1.63679)
1,Uganda - Kicalaca-Kanyatete - 1022733,Kabarole,Kicalaca-Kanyatete,1022733,Identified,Identified in Needs Assessment,2020,1820.0,NaN,POINT (30.29366 0.72535)
2,Uganda - Kakonkome - 1022735,Kabarole,Kakonkome,1022735,Identified,Identified in Needs Assessment,2020,7360.0,NaN,POINT (30.28677 0.68186)
3,Uganda - Magunga- Nyakagongo - 1022726,Kabarole,Magunga- Nyakagongo,1022726,Identified,Identified in Needs Assessment,2021,11150.0,NaN,POINT (30.20300 0.59040)
4,Uganda - Kikonge-Bwanika - 1022734,Kabarole,Kikonge-Bwanika,1022734,Identified,Identified in Needs Assessment,2020,4660.0,NaN,POINT (30.26291 0.70312)
...,...,...,...,...,...,...,...,...,...,...
585,Uganda - Koibei - 1011063,Bukwa,Koibei,1011063,Rejected,Technical,2017,1400.0,3417.0,POINT (34.76456 1.25509)
586,Uganda - Sosho - 1011065,Bukwa,Sosho,1011065,Identified,Identified in Needs Assessment,2017,5300.0,7422.0,POINT (34.71863 1.28062)
587,Uganda - Kapnandi Bridge - 1011060,Bukwa,Kapnandi Bridge,1011060,Identified,Identified in Needs Assessment,2017,28000.0,657.0,POINT (34.80482 1.22772)
588,Uganda - Mwongera 2- kakari 2 - 1022224,Rubirizi,Mwongera 2- kakari 2,1022224,Identified,Identified in Needs Assessment,2020,2783.0,NaN,POINT (30.20009 -0.18291)


In the EPSG 4326 coordinate system, 1 degree = 69 miles. We want to calculate a 2 mile buffer around each point. 2 miles / 69 miles per degree ≈ 0.0289855072463768 degrees

In [9]:
uga_bridges['geometry'] = uga_bridges['geometry'].buffer(0.029)

/var/folders/3v/8xxt4zw96bn2l6qwb8w7_5r40000gn/T/ipykernel_66625/58340433.py:1: UserWarning: Geometry is in a geographic CRS. Results from 'buffer' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  uga_bridges['geometry'] = uga_bridges['geometry'].buffer(0.029)


Population data was procured from World Pop 2020 data. This was converted from raster to vector points using QGIS raster pixels to points. 

In [10]:
# read in rwa_pop_2020.geojson
uga_pop = gpd.read_file('unsynced-data/uga-pop-2020.geojson')

In [16]:
# calculate the sum of rwa_pop points within each of the rwa_bridges polygons and add this to a new column in uga_bridges
uga_bridges['pop_sum'] = uga_bridges['geometry'].apply(lambda x: uga_pop['population'].loc[uga_pop.within(x)].sum())
uga_bridges['pop_sum'] = uga_bridges['pop_sum'].astype(int)

/opt/homebrew/Caskroom/miniforge/base/envs/geo/lib/python3.10/site-packages/geopandas/geodataframe.py:1443: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  super().__setitem__(key, value)
/opt/homebrew/Caskroom/miniforge/base/envs/geo/lib/python3.10/site-packages/geopandas/geodataframe.py:1443: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  super().__setitem__(key, value)


In [17]:
# Previewing data
uga_bridges

,Opportunity Name,Level 2 Government,Bridge Name,Project Code,Stage,Sub-Stage,B2P Fiscal Year,Individuals Directly Served,Population Estimate 2000m,geometry,pop_sum
0,- Kikambajo - 1037598,Hoima,Kikambajo,1037598,Identified,NaN,2021,NaN,NaN,"POLYGON ((31.36257 1.63679, 31.36243 1.63395, ...",15539
1,Uganda - Kicalaca-Kanyatete - 1022733,Kabarole,Kicalaca-Kanyatete,1022733,Identified,Identified in Needs Assessment,2020,1820.0,NaN,"POLYGON ((30.32266 0.72535, 30.32252 0.72251, ...",9687
2,Uganda - Kakonkome - 1022735,Kabarole,Kakonkome,1022735,Identified,Identified in Needs Assessment,2020,7360.0,NaN,"POLYGON ((30.31577 0.68186, 30.31564 0.67902, ...",33269
3,Uganda - Magunga- Nyakagongo - 1022726,Kabarole,Magunga- Nyakagongo,1022726,Identified,Identified in Needs Assessment,2021,11150.0,NaN,"POLYGON ((30.23200 0.59040, 30.23186 0.58755, ...",21920
4,Uganda - Kikonge-Bwanika - 1022734,Kabarole,Kikonge-Bwanika,1022734,Identified,Identified in Needs Assessment,2020,4660.0,NaN,"POLYGON ((30.29191 0.70312, 30.29177 0.70028, ...",15678
...,...,...,...,...,...,...,...,...,...,...,...
585,Uganda - Koibei - 1011063,Bukwa,Koibei,1011063,Rejected,Technical,2017,1400.0,3417.0,"POLYGON ((34.79356 1.25509, 34.79342 1.25224, ...",16929
586,Uganda - Sosho - 1011065,Bukwa,Sosho,1011065,Identified,Identified in Needs Assessment,2017,5300.0,7422.0,"POLYGON ((34.74763 1.28062, 34.74749 1.27778, ...",21103
587,Uganda - Kapnandi Bridge - 1011060,Bukwa,Kapnandi Bridge,1011060,Identified,Identified in Needs Assessment,2017,28000.0,657.0,"POLYGON ((34.83382 1.22772, 34.83368 1.22488, ...",4393
588,Uganda - Mwongera 2- kakari 2 - 1022224,Rubirizi,Mwongera 2- kakari 2,1022224,Identified,Identified in Needs Assessment,2020,2783.0,NaN,"POLYGON ((30.22909 -0.18291, 30.22895 -0.18575...",11352


In [18]:
uga_bridges.explore()

In [19]:
uga_bridges.to_file('data/uga-population-2mi.geojson', driver='GeoJSON')
uga_bridges.to_csv('data/uga-population-2mi.csv')